In [1]:
!git clone https://github.com/RUCAIBox/HaluEval.git

Cloning into 'HaluEval'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 166 (delta 37), reused 33 (delta 9), pack-reused 70
Receiving objects: 100% (166/166), 37.92 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00


In [3]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pickle
# import graphvite as gv
#import dataset
import nltk
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
!pip install transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [5]:
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login

notebook_login()

In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [7]:
!wget -O processed_subgraphs.pt https://www.dropbox.com/scl/fi/py1xg4a4a88mcegw3umve/processed_subgraphs.pt?rlkey=ffks4lfvcj2sttk0i1qjwr97l&dl=0

--2023-11-29 23:48:42--  https://www.dropbox.com/scl/fi/py1xg4a4a88mcegw3umve/processed_subgraphs.pt?rlkey=ffks4lfvcj2sttk0i1qjwr97l
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8313edb045d64b18c906b43caa.dl.dropboxusercontent.com/cd/0/inline/CIck5s0ugVorR5tBSdmvt5L2IHwx49N8upqD81x34yWaoY-jCReCBzIP1y2Es_ePhaD17sZf1_DsnBAHvS_cbFTqKWx19ZnbxeJVl81AJUGfnnJ2Y6MQvW65jd-k_w_eAdsn6D7IFu8C49bYgs621dEF/file# [following]
--2023-11-29 23:48:42--  https://uc8313edb045d64b18c906b43caa.dl.dropboxusercontent.com/cd/0/inline/CIck5s0ugVorR5tBSdmvt5L2IHwx49N8upqD81x34yWaoY-jCReCBzIP1y2Es_ePhaD17sZf1_DsnBAHvS_cbFTqKWx19ZnbxeJVl81AJUGfnnJ2Y6MQvW65jd-k_w_eAdsn6D7IFu8C49bYgs621dEF/file
Resolving uc8313edb045d64b18c906b43caa.dl.dropboxusercontent.com (uc8313edb045d64b18c906b43caa.dl.dropboxusercontent.com)... 162.125.3.

In [8]:
import torch

graph_data = torch.load("processed_subgraphs.pt")

In [9]:
len(graph_data)

10000

In [10]:
import json

# Opening JSON file
data_list = []
with open('HaluEval/data/qa_data.json') as json_file:
    for jsonObj in json_file:
        data_dict = json.loads(jsonObj)
        data_list.append(data_dict)



In [11]:
connection_dict = {obj["question"].strip().lower() : i for i, obj in enumerate(data_list)}

In [12]:
len(connection_dict)

10000

In [13]:
from datasets import load_dataset

def get_data(data_path):

  ds = load_dataset("json", data_files=data_path, split="train")
  ds = ds.train_test_split(test_size=0.2)
  return ds

#print(os.listdir())

ds = get_data("HaluEval/data/qa_data.json")
print(ds)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 2000
    })
})


In [14]:
import torch
from torch_geometric.data import Batch

def preprocess_function(examples):
  questions = [q.strip().lower() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      [c.lower() for c in examples["knowledge"]],
      max_length=384,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
  )
  offset_mapping = inputs.pop("offset_mapping")
  answers = examples["right_answer"]
  start_positions = []
  end_positions = []
  graph_embeddings = []

  for i, offset in enumerate(offset_mapping):
      answer = answers[i]
      start_char = examples["knowledge"][i].find(answer)
      end_char = start_char + len(answer)
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

      corr_index = connection_dict[questions[i]]
      graph_index = graph_data[corr_index]
      token_list = inputs.input_ids[i]
      graph_embeddings.append(graph_index)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  #batch = Batch.from_data_list([graph_index for graph_index in graph_embeddings])
  inputs["x"] = []
  inputs["edge_index"] = []
  inputs["edge_attr"] = []
  for graph_index in graph_embeddings:
      inputs["x"].append(graph_index.x)
      inputs["edge_index"].append(graph_index.edge_index)
      inputs["edge_attr"].append(graph_index.edge_attr)

  return inputs

In [15]:
ds["train"][1]

{'knowledge': "The 29th Annual Kids' Choice Awards was held live on March 12, 2016, at The Forum in Inglewood, California live on Nickelodeon and either live or on tape delay across all of Nickelodeon's international networks and also served as a full-length TV movie. Opening on December 30, 1967, the Forum was an unusual and groundbreaking structure.",
 'question': "When did the venue of the The 29th Annual Kids' Choice Awards first open?",
 'right_answer': 'December 30, 1967',
 'hallucinated_answer': "The venue of The 29th Annual Kids' Choice Awards first opened on December 31, 1967."}

In [16]:
example = {
    "knowledge": [ds["train"][1]["knowledge"]],
    "question": [ds["train"][1]["question"]],
    "right_answer": [ds["train"][1]["right_answer"]],
    "hallucinated_answer": [ds["train"][1]["hallucinated_answer"]]
}
#print(clean_text(example))
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = preprocess_function(example)
print(inputs)
print(tokenizer.decode(inputs["input_ids"][0]))

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

{'input_ids': [[0, 14746, 222, 5, 5584, 9, 5, 5, 1132, 212, 1013, 1159, 108, 2031, 4188, 78, 490, 116, 2, 2, 627, 1132, 212, 1013, 1159, 108, 2031, 4188, 21, 547, 697, 15, 6674, 316, 6, 336, 6, 23, 5, 7900, 11, 21691, 459, 1845, 6, 13011, 1594, 43052, 697, 15, 19500, 4636, 261, 8, 1169, 697, 50, 15, 7898, 4646, 420, 70, 9, 19500, 4636, 261, 18, 758, 4836, 8, 67, 1665, 25, 10, 455, 12, 16096, 30016, 1569, 4, 1273, 15, 263, 47153, 389, 6, 13025, 6, 5, 7900, 21, 41, 5425, 8, 18910, 3184, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [17]:
inputs["batch"].shape

KeyError: ignored

In [18]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_ds = ds.map(preprocess_function, batched=True, batch_size=50, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [19]:
print(len(tokenized_ds["train"][0]['input_ids']))
print(len(tokenized_ds["train"][0]['attention_mask']))
print(tokenized_ds["train"][0]['start_positions'])
print(tokenized_ds["train"][0]['end_positions'])
#print(tokenized_ds["train"][0]["graph_embeddings"])
#print(len(tokenized_ds["train"][0]["graph_embeddings"]))
#print(len(tokenized_ds["train"][0]["graph_embeddings"][0]))
#print(tokenized_ds["train"][0:10])

384
384
49
52


In [89]:
def collate_function(batch):
  input_ids = torch.stack([torch.tensor(example["input_ids"]) for example in batch])
  attention_mask = torch.stack([torch.tensor(example["attention_mask"]) for example in batch])
  start_positions = torch.tensor([example["start_positions"] for example in batch])
  end_positions = torch.tensor([example["end_positions"] for example in batch])
  batch_size = len(batch)

  data_list = []
  for i in range(batch_size):
    data_list.append(Data(x=torch.tensor(batch[i]["x"]), edge_index=torch.tensor(batch[i]["edge_index"]), edge_attr=torch.tensor(batch[i]["edge_attr"])))
  batch = Batch.from_data_list(data_list)

  return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "start_positions": start_positions,
        "end_positions": end_positions,
        "graph_batch": batch
      }

In [64]:
collate_function(tokenized_ds["train"][0:10])

[49, 82, 66, 69, 123, 110, 52, 30, 31, 146]


{'input_ids': tensor([[    0,  5488,     5,  ...,     1,     1,     1],
         [    0, 14746,   222,  ...,     1,     1,     1],
         [    0,   642,  6695,  ...,     1,     1,     1],
         ...,
         [    0, 12196,  3390,  ...,     1,     1,     1],
         [    0,  8155, 15266,  ...,     1,     1,     1],
         [    0, 14170, 22501,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'start_positions': tensor([ 49,  82,  66,  69, 123, 110,  52,  30,  31, 146]),
 'end_positions': tensor([ 52,  86,  69,  70, 126, 111,  55,  35,  33, 152]),
 'graph_batch': DataBatch(x=[1608, 384], edge_index=[2, 6675], edge_attr=[6675, 1], batch=[1608], ptr=[11])}

In [21]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

class GraphConvModel(torch.nn.Module):
    def __init__(self):
        super(GraphConvModel, self).__init__()

        self.initial_features = 384
        self.hidden_size = 384
        self.graph_embedding_size = 768
        self.gat = GATConv(self.initial_features, self.hidden_size, edge_dim=1)
        self.lin = Linear(self.hidden_size, self.graph_embedding_size)

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.gat(x, edge_index, edge_attr)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = self.lin(x)

        return x

In [68]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import QuestionAnsweringModelOutput
from torch_geometric.data import Batch, Data

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.sentence_size = 384
    self.graph_conv = GraphConvModel()
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)
    self.fc = nn.Linear(2 * self.llm_embedding_size, self.llm_embedding_size)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

    for param in self.llm.qa_outputs.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions, graph_batch):
     outputs = self.llm.roberta(input_ids, attention_mask)
     sequence_outputs = outputs.last_hidden_state

     graph_embedding = self.graph_conv(graph_batch.x, graph_batch.edge_index, graph_batch.edge_attr, graph_batch.batch)

     enriched_encodings = self.fc(torch.cat((sequence_outputs, torch.stack([graph_embedding for i in range(self.sentence_size)], 1)), 2))
     enriched_encodings = F.relu(enriched_encodings)

     logits = self.llm.qa_outputs(enriched_encodings)
     start_logits, end_logits = logits.split(1, dim=-1)
     start_logits = start_logits.squeeze(-1).contiguous()
     end_logits = end_logits.squeeze(-1).contiguous()

     total_loss = None
     if start_positions is not None and end_positions is not None:
          # If we are on multi-GPU, split add a dimension
         if len(start_positions.size()) > 1:
             start_positions = start_positions.squeeze(-1)
         if len(end_positions.size()) > 1:
             end_positions = end_positions.squeeze(-1)
          # sometimes the start/end positions are outside our model inputs, we ignore these terms
         ignored_index = start_logits.size(1)
         start_positions = start_positions.clamp(0, ignored_index)
         end_positions = end_positions.clamp(0, ignored_index)

         loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
         start_loss = loss_fct(start_logits, start_positions)
         end_loss = loss_fct(end_logits, end_positions)
         total_loss = (start_loss + end_loss) / 2

     return QuestionAnsweringModelOutput(
         loss=total_loss,
         start_logits=start_logits,
         end_logits=end_logits,
         hidden_states=outputs.hidden_states,
         attentions=outputs.attentions,
     )

In [69]:
model = GraphEnrichedRoberta(model_name)

In [70]:
with torch.no_grad():
  input_ids_test = torch.tensor(tokenized_ds["train"][0]["input_ids"]).unsqueeze(0)
  attn_mask_test = torch.tensor(tokenized_ds["train"][0]["attention_mask"]).unsqueeze(0)
  start_positions_test = torch.tensor(tokenized_ds["train"][0]["start_positions"]).unsqueeze(0)
  end_positions_test = torch.tensor(tokenized_ds["train"][0]["end_positions"]).unsqueeze(0)
  x_test = torch.tensor(tokenized_ds["train"][0]["x"]).unsqueeze(0)
  edge_index_test = torch.tensor(tokenized_ds["train"][0]["edge_index"]).unsqueeze(0)
  edge_attr_test = torch.tensor(tokenized_ds["train"][0]["edge_attr"]).unsqueeze(0)
  batch_vals = collate_function({
      "input_ids": input_ids_test,
      "attention_mask": attn_mask_test,
      "start_positions": start_positions_test,
      "end_positions": start_positions_test,
      "x": x_test,
      "edge_index": edge_index_test,
      "edge_attr": edge_attr_test
  })
  print(model(**batch_vals))

tensor([49])


<ipython-input-63-d33678b8b87c>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch["input_ids"])
<ipython-input-63-d33678b8b87c>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch["attention_mask"])
<ipython-input-63-d33678b8b87c>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_positions = torch.tensor(batch["start_positions"])
<ipython-input-63-d33678b8b87c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourc

QuestionAnsweringModelOutput(loss=tensor(6.2055), start_logits=tensor([[-0.1298, -0.1515, -0.1248, -0.1512, -0.1398, -0.0959, -0.1964, -0.1012,
         -0.1184, -0.0933, -0.1433, -0.0870, -0.0895, -0.1654, -0.1086, -0.0687,
         -0.0436, -0.1189, -0.1259, -0.1269, -0.0681, -0.0956, -0.1111, -0.1814,
         -0.1927, -0.1056, -0.1681, -0.1724, -0.1465, -0.1364, -0.0860, -0.1216,
         -0.0982, -0.1393, -0.1442, -0.1328, -0.1346, -0.1830, -0.1329, -0.0956,
         -0.1451, -0.1260, -0.1482, -0.1515, -0.0918, -0.1572, -0.1397, -0.2380,
         -0.1114, -0.2538, -0.2216, -0.1583, -0.0654, -0.0812, -0.1807, -0.1198,
         -0.1667, -0.1396, -0.1415, -0.0891, -0.1450, -0.1340, -0.0683, -0.1879,
         -0.1195, -0.1017, -0.0845, -0.1466, -0.1001, -0.1427, -0.1057, -0.2435,
         -0.1930, -0.2937, -0.2272, -0.2448, -0.0813, -0.1218, -0.0781, -0.1798,
         -0.1684, -0.1380, -0.1014, -0.0793, -0.1392, -0.1427, -0.1405, -0.1719,
         -0.0923, -0.0221, -0.0648, -0.0627, -

In [71]:
print(model)

GraphEnrichedRoberta(
  (graph_conv): GraphConvModel(
    (gat): GATConv(384, 384, heads=1)
    (lin): Linear(in_features=384, out_features=768, bias=True)
  )
  (llm): RobertaForQuestionAnswering(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
   

In [110]:
training_args = TrainingArguments(
    output_dir="roberta_qa_model_with_gnn",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    remove_unused_columns=False,
    logging_steps=250
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=collate_function,
)

In [108]:
from datasets.arrow_dataset import Dataset
type(Dataset.from_dict(tokenized_ds["train"][:50])) # <- use this command to reduce size of train/eval datasets if necessary

datasets.arrow_dataset.Dataset

In [111]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.692100,No log
2,0.998000,No log
3,1.374400,No log


TrainOutput(global_step=12, training_loss=1.387604186932246, metrics={'train_runtime': 88.2071, 'train_samples_per_second': 1.701, 'train_steps_per_second': 0.136, 'total_flos': 0.0, 'train_loss': 1.387604186932246, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), 'gnn_llm_model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('gnn_llm_weights.pth'))

<All keys matched successfully>

In [ ]:
model

GraphEnrichedRoberta(
  (llm): RobertaForQuestionAnswering(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): 

In [ ]:
#tokenized_ds["test"]

predictions = trainer.predict(tokenized_ds["test"])

In [ ]:
import numpy as np

for i in range(len(predictions.predictions)):
  print(predictions.predictions[i].shape)

print(predictions.predictions[0])

print(np.array(tokenized_ds["test"]["start_positions"]).shape)


(125,)
(2000, 384)
(2000, 384)
[1.0641084  1.6659701  1.9759023  1.2464397  1.970095   1.2182294
 1.7953851  1.3778882  1.089099   2.3120296  1.0138576  1.7308497
 1.5308025  1.2966044  1.4876668  0.62224865 2.2005458  1.19306
 1.9669992  1.8536528  2.293821   1.792011   0.7256372  1.629729
 2.1631832  1.4701074  1.4578855  1.6019956  1.2260633  1.738751
 2.4187698  1.3923588  1.9579014  1.3781996  2.456057   1.0412458
 1.7440045  1.8541543  2.8645754  0.93838847 1.7694396  1.6841547
 1.9321573  1.4411316  1.38648    1.194206   0.81366175 1.0065814
 0.8059305  1.9596742  0.79150534 1.9571393  0.9703902  1.6299062
 1.890945   1.146168   1.0196122  1.1374956  2.0560458  1.7159839
 1.968475   1.3967826  1.6296978  1.5666738  1.2158916  1.7590964
 2.4377935  1.9311898  1.9091766  2.4590755  1.9080064  1.0684565
 1.7132566  2.3912418  1.6143143  1.9749919  1.1747274  1.9486256
 1.5132129  1.6976893  1.2747147  1.1367294  2.944383   1.1048306
 1.9857776  1.3243711  1.0923741  1.5487192  1.53

In [ ]:
avg_loss = np.mean(predictions.predictions[0])
avg_loss

1.6121981

In [ ]:
start_logits = predictions.predictions[1]
end_logits = predictions.predictions[2]

start_positions = np.array(tokenized_ds["test"]["start_positions"])
end_positions = np.array(tokenized_ds["test"]["end_positions"])

In [ ]:
start_positions_pred = np.argmax(start_logits, axis=1)
end_positions_pred = np.argmax(end_logits, axis=1)

In [ ]:
print(start_positions_pred)
print(start_positions)
print(end_positions_pred)
print(end_positions)
print(np.sum(start_positions_pred == start_positions))
overlap = np.maximum(0.0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
print(overlap)
val1 = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
val2 = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
print(np.min(val1))
print(np.max(val1))
print(np.min(val2))
print(np.max(val2))

[ 39  55  97 ... 127  51  24]
[ 39  55  97 ... 127  51  24]
[ 42  57 102 ... 130  56  29]
[ 42  57 102 ... 130  56  29]
1165
[4. 3. 6. ... 4. 6. 6.]
0.0
1.0
0.0
1.0


In [ ]:
def exact_match(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  same_start = start_positions_pred == start_positions
  same_end = end_positions_pred == end_positions
  return np.sum(np.logical_and(same_start, same_end)) / start_positions.shape[0]

In [ ]:
exact_match(start_logits, end_logits, start_positions, end_positions)

0.505

In [ ]:
def f1_score(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  overlap = np.maximum(0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
  recall = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
  precision = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
  f1 = (2 * precision * recall) / (precision + recall + 0.00000001)
  return np.mean(f1)

In [ ]:
f1_score(start_logits, end_logits, start_positions, end_positions)

0.6002186955899961

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions):
    return self.llm(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

In [ ]:
model = GraphEnrichedRoberta(model_name)

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>